**Importing Modules**

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

**Loading Dataset**

In [4]:
BASE_DIR = "../dataset/"

In [5]:
# df = pd.read_csv(os.path.join(BASE_DIR,"train.csv","train.csv"))

In [ ]:
df.head()

**Preprocess**

In [7]:
from tensorflow.keras.layers import TextVectorization

In [8]:
X = df["comment_text"]
y = df[df.columns[2:]].values

In [14]:
MAX_FEATURES = 200000

In [15]:
vectorizer = TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=1800,
    output_mode="int"
)

In [16]:
vectorizer.adapt(X.values)

In [17]:
vectorized_text = vectorizer(X.values)

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [22]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

**Create Sequential Model**

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding

In [24]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1,32))
model.add(Bidirectional(LSTM(32,activation="tanh")))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [25]:
model.compile(loss="BinaryCrossentropy",optimizer="Adam")

In [ ]:
model.summary()

In [ ]:
history = model.fit(train,epochs=1,validation_data=val)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figure=(8,5))
pd.DataFrame(history.history).plot()
plt.show

**Make Predictions**

In [ ]:
input_text = vectorizer("I Love You!!")

In [ ]:
res = model.predict(input_text)

In [ ]:
(res > 0.5).astype(int)

In [ ]:
batch_X,batch_y = test.as_numpy_iterator().next

In [ ]:
(model.predict(batch_X) > 0.5).astype(int)

In [ ]:
res.shape

**Evaluate Model**

In [ ]:
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall
from tensorflow.keras.metrics import CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X_true, y_true = batch
    yhat = model.predict(X_true)
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

**Test And Gradio**

In [ ]:
import tensorflow as tf
import gradio as gr

In [ ]:
model.save("model.h5")

In [ ]:
model = tf.keras.models.load_model("model.h5")

In [ ]:
input_str = vectorizer("I Hate You")

In [ ]:
res = model.predict(np.expand_dims(input_str,0))

In [ ]:
res

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)